In [ ]:
# pip install librosa
# pip install keras
# pip install tensorflow
# pip install os

In [3]:
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical


import module.lstm as LSTM_MODEL
import module.preprocess as PROCESSING

In [4]:
encoder = LabelEncoder()
SR = 16000

ROOT = "C:/Users/SSAFY/Desktop/S08P22A603/AI"
TRAIN_PATH = f"{ROOT}/train"
VAL_PATH = f"{ROOT}/val"
TENSER_PATH = f"{ROOT}/tensor"
CHECKPOINT_PATH = f"{ROOT}/checkpoint"

In [5]:
# train_mfccs, train_labels = PROCESSING.createDATA(TRAIN_PATH,SR)
train_mfccs, train_labels = np.load(f'{TENSER_PATH}/train_mfccs.npy'), np.load(f'{TENSER_PATH}/train_labels.npy')

In [6]:
#val_mfccs, val_labels = PROCESSING.createDATA(VAL_PATH,SR)
val_mfccs, val_labels = np.load(f'{TENSER_PATH}/val_mfccs.npy'), np.load(f'{TENSER_PATH}/val_labels.npy')

In [8]:
# np.save(f'{TENSER_PATH}/train_mfccs.npy', train_mfccs)
# np.save(f'{TENSER_PATH}/train_labels.npy', train_labels)
# np.save(f'{TENSER_PATH}/val_mfccs.npy', val_mfccs)
# np.save(f'{TENSER_PATH}/val_labels.npy', val_labels)
# np.save(f'{TENSER_PATH}/label.npy',np.concatenate([train_labels, val_labels]))

In [7]:
encoder.fit(np.concatenate([train_labels, val_labels]))
train_y = encoder.transform(train_labels)
val_y = encoder.transform(val_labels)

In [ ]:
np.save(f'{TENSER_PATH}/label.npy',np.concatenate([train_labels, val_labels]))

In [9]:
CHECK_POINT_FILE = "only_mfcc/checkpoint.h5"
model = LSTM_MODEL.create_lstm_model(len(encoder.classes_),sr = SR)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
checkpoint_callback = LSTM_MODEL.Checkpoint(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")


if os.path.isfile(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}"):
    model.load_weights(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")


In [ ]:
model.fit(train_mfccs, to_categorical(train_y,num_classes=len(encoder.classes_)), epochs=50, validation_data=(val_mfccs, to_categorical(val_y,num_classes=len(encoder.classes_))), callbacks=[checkpoint_callback], initial_epoch=0)
#model.save(f"{CHECKPOINT_PATH}/{CHECK_POINT_FILE}")

In [12]:
test_wav = PROCESSING.load_wav_file(f"{ROOT}/sample/윤하.wav")
test_wav = PROCESSING.preprocess_mfccs([test_wav])

In [13]:
pred = model.predict(test_wav)

1/1 [==============================] - 1s 1s/step


In [16]:
pred

array([[1.7746225e-04, 8.7304093e-04, 4.6626059e-03, 1.3746046e-04,
        3.1145032e-02, 4.5432109e-05, 3.4007855e-06, 8.9329149e-08,
        4.3814158e-08, 8.4339752e-07, 3.3166647e-02, 7.9781739e-06,
        6.5324849e-07, 3.0410013e-03, 3.8972557e-05, 4.1699060e-03,
        5.0002490e-07, 1.4452484e-03, 9.6086165e-05, 9.0644602e-04,
        1.4859433e-05, 6.8549169e-05, 1.6056040e-06, 1.3180203e-07,
        1.9910496e-02, 2.7217731e-02, 7.4066827e-04, 8.2392397e-04,
        9.5675616e-03, 1.1337686e-06, 2.4621119e-03, 3.8619040e-05,
        2.2042093e-06, 2.1316602e-03, 7.7166106e-07, 1.2876929e-01,
        2.2624934e-07, 1.4878534e-05, 4.4595328e-07, 3.3864630e-03,
        1.6536128e-02, 1.2074690e-05, 1.5152334e-01, 2.8180912e-01,
        3.9944044e-09, 1.9776842e-06, 6.0246348e-02, 5.1618017e-06,
        9.5841619e-07, 1.1315948e-05, 2.0475026e-02, 6.7439497e-02,
        2.1392994e-03, 3.9971810e-06, 2.0446339e-05, 5.6093495e-04,
        1.0836242e-05, 1.4465237e-08, 3.9393863e

In [14]:
pred = model.predict(test_wav)
top_5 = np.argsort(pred, axis=1)[:, -5:]
top_5_labels = encoder.inverse_transform(top_5.T)

1/1 [==============================] - 0s 394ms/step


c:\Users\SSAFY\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [15]:
top_5_labels

array(['SINGER_47', 'SINGER_52', 'SINGER_36', 'SINGER_43', 'SINGER_44'],
      dtype='<U9')

In [ ]:
pred